In [1]:
import csv
import pickle
import numpy as np
import os
import tempfile
import tensorflow_hub as hub

C:\Users\conta\git-CVC\Formation\Python\RAG\git-rag\myvenv\Lib\site-packages\tensorflow_hub\__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


In [2]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #"https://tfhub.dev/google/universal-sentence-encoder-lite/2" #"https://tfhub.dev/google/universal-sentence-encoder/4" #"https://tfhub.dev/google/universal-sentence-encoder-large/5"
model = hub.load(module_url)
print(f"module {module_url} loaded")

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [3]:
class Item:

    def __init__(self, question, answer):
        self.question = question
        self.answer = answer
        self.embed = None


class UseService:

    def __init__(self):
        self.items: list[Item] = []

    def embed(self, s: str):
        return model([s])[0].numpy()

    def score(self, l1: list[float], l2: list[float]) -> float:
        return np.inner(l1, l2)

    def train(self, path: str):
        print("Training")
        with open(path, encoding="utf-8") as f:
            reader = csv.DictReader(f, delimiter="|")
            for row in reader:
                q = row["question"]
                r = row["answer"]
                if r is not None and r.strip() != "":
                    item = Item(q, r)
                    item.embed = self.embed(q)
                    self.items.append(item)
        print("\nSaving")
        with open(path.replace(".txt", "_model.pickle"), "wb") as f:
            pickle.dump(self.items, f)

    def load_model(self, path):
        print("Load model")
        with open(path, "rb") as f:
            self.items = pickle.load(f)

    def predict(self, s: str):
        embed = self.embed(s)
        best_score = 0
        best_item = None
        for item in self.items:
            score = self.score(embed, item.embed)
            if score > best_score:
                best_score = score
                best_item = item
                if best_score > 0.99:
                    break
        return best_item, best_score


In [ ]:
service = UseService()
service.train("data/chatbot/dialogs_fr.txt")
service.load_model("data/chatbot/dialogs_fr_model.pickle")
print("> Bonjour")
res = service.predict("Bonjour")
print(f"{res[0].answer} @{res[1] * 100:.0f}%")
while True:
    s = input("> ")
    res = service.predict(s)
    print(f"{res[0].answer} @{res[1] * 100:.0f}%")

Training

Saving
Load model
> Bonjour
Comment allez vous? @100%


>  Je vais bien et vous ?


Je suis assez bien. Merci d'avoir posé la question. @98%


>  je vais au PCC.


est-ce que tu aimes ça là-bas ?  @100%


>  Comment faire un bon 30/30


tu vois tous ces cheveux gris? c'était totalement noir il y a cinq ans. @46%
